# Proyecto: Análisis del mercado inmobiliario de la Ciudad de Buenos Aires

Mentor: Javier Lezama.

javitolez@gmail.com

En este proyecto vamos a trabajar con un dataset de propiedades en venta publicadas en el portal www.Properati.com.ar.

# El dataset: propiedades en venta en Ciudad de Buenos Aires.

En este proyecto, trabajaremos con una muestra del conjunto de datos de propiedades en venta mencionado. En este dataset, cada fila es una propiedad en venta. 

#Los atributos

A continuación vamos a describir los atributos que consideramos en esta muestra:
* id: id de la propiedad
* created_on: fecha en la que la propiedad ingresó al sitio
* operation: alquiler (rent) o venta (sell)
* property_type: tipo de propiedad (casa, departamento, ph, etcétera)
* place_with_parent_names: nombre del lugar donde se encuentra la propiedad según el publicador
* lat-lon: coordenadas concatenadas
* lat: latitud
* lon: longitud
* price: precio en la moneda especificada en currency
* currency: divisa en la que está expresada la publicación
* price_aprox_usd: precio aproximado en dólares estadounidenses
* surface_total_in_m2: superficie total (en metros cuadrados)
* surface_covered_in_m2: superficie cubierta (en metros cuadrados)
* price_usd_per_m2: precio por metro cuadrado en dólares (precio dólares / superficie)
* floor: número de piso (si corresponde)
* rooms: cantidad de ambientes
* expenses: expensas (si corresponde)
* barrio: barrio según cartografía oficial
* properati_url: url de la publicación en Properati

Objetivos del practico introducción al aprendizaje automático:

El objetivo de este practico es trabajar con el mismo dateset trabajado en el práctico dos, específicamente el objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

Para eso trabajaremos con el dataset resultante del practico dos para armar nuestros primeros modelos para predecir el precio de las propiedades en dólares.

# Manos a la obra

Partiremos con el dataset resultante del práctico dos. El práctico está basado en el práctico final de la materia introducción al aprendizaje automatico del año 2018.

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib.colors import ListedColormap
from sklearn.datasets import load_boston, load_breast_cancer, load_iris
from sklearn import linear_model
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

np.random.seed(1234)  # Setup seed to be more deterministic

%matplotlib inline

In [2]:
df_prop = pd.read_csv('df_bin.csv')
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7716 entries, 0 to 7715
Data columns (total 79 columns):
Unnamed: 0                                                                       7716 non-null int64
created_on                                                                       7716 non-null object
operation                                                                        7716 non-null object
property_type                                                                    7716 non-null int64
place_with_parent_names                                                          7716 non-null object
lat-lon                                                                          7716 non-null object
lat                                                                              7716 non-null float64
lon                                                                              7716 non-null float64
price_aprox_usd                                                              

# Ejercicio 1: División de datos en conjuntos de entrenamiento y evaluación.

La primer tarea consiste en dividir el conjunto de datos cargados en el apartado anterior en conjuntos de entrenamiento (o training) y evaluación (o test).

Se define un dataframe con los features que se consideran importantes para la definición del precio de la propiedad

In [3]:
df_prop_gt = df_prop[['surface_total_in_m2','barrio_cod','property_type','price_aprox_usd']]
df_prop_gt.head()

,surface_total_in_m2,barrio_cod,property_type,price_aprox_usd
0,130.0,20,1,340000.00
1,70.0,20,1,180000.00
2,532.0,20,2,1200000.00
3,72.0,20,0,219000.00
4,40.0,20,1,119124.77


Se separa el set de datos tomando el 80% para entrenamiento y el 20% para validación

In [4]:
#Utilizamos aproximadamente 90% de los datos para entrenamiento y 20% para validación
#entradas = len(df_prop)
#data_div = round(entradas*0.9)

#shuff_data = np.random.permutation(506)
#shuff_train = shuff_data[:data_div]
#shuff_val = shuff_data[data_div:]

X = df_prop_gt.drop('price_aprox_usd', axis=1)
y = df_prop_gt.price_aprox_usd

from sklearn.model_selection import train_test_split
#Random state sirve para sembrar una "Seed" de como se distribuyen los datos para el algoritmo
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Necesario para poder hacer un regresor por feature
#features_names = df_prop.columns
#feature_map = {feature: idx for idx, feature in enumerate(features_names)}


# Ejercicio 2: Elección de un modelo de regresión.

https://gist.github.com/lprowell/81ab813d84096290c5a833a8c7731355

Basándose en lo visto en el teórico escojan y justifiquen un modelo de aprendizaje automático de regresión, para predecir el precio de las propiedades en dólares. Selección de regulizador, selección de la función de coste, etc.

- Elección Modelo Regresión Lineal

In [33]:
# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
#model = LinearRegression()
model = linear_model.Ridge(alpha=0.01)
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
#print('Media del error cuadrado para validación: %.2f' %
#      mean_squared_error(yy, model.predict(XX)))

Media del error cuadrado para entrenamiento: 1386185771.41


- Elección Modelo Regresión Polinomial

In [15]:
polynomial_degree = 3 # TODO: Grado del polinomio.
alpha = 0.1 #TODO: Parámetro de regularización. También denominado como parámetro `lambda`.

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(X_train)
X_poly_train = poly_features.transform(X_train)
X_poly_val = poly_features.transform(X_valid)

model = Ridge(alpha=alpha)
model.fit(X_poly_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
#print('Media del error cuadrado para validación: %.2f' %
#      mean_squared_error(y_val, model.predict(X_poly_val)))

Media del error cuadrado para entrenamiento: 1201759627.81


C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.903735e-26
  overwrite_a=True).T


# Ejercicio 3: Selección de hiperparámetros.

Utilizando búsqueda exhaustiva (grid search) con 5-fold cross-validation y utilizando como métrica el área bajo la curva de ROC (o ROC-AUC), hagan una selección de los mejores hiperparámetros para su conjunto de datos y el modelo que hayan elegido en el apartado anterior.

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### GridSearchCV Regresión Lineal

In [35]:
params={'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
estimators = linear_model.Lasso()

model = GridSearchCV(estimators,params,cv=5,verbose = 1, scoring = 'neg_mean_squared_error')         
model.fit(X_train, y_train)

#Acá arme un dataframe con los resultados, pero la clave sería ver el reporte que armó el profe en clase 
pd.DataFrame(model.cv_results_)

#print("# Exploración de hiperparámetros para función de coste \"%s\"" % loss, end="\n\n")
    
#print("Mejor conjunto de parámetros:")
#print(model.best_params_, end="\n\n")



Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  65 out of  65 | elapsed:    0.4s finished
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.002587,0.001846,0.002560,0.001953,25,{'alpha': 25},-1.275752e+09,-1.426342e+09,-1.348247e+09,-1.398785e+09,...,-1.388583e+09,7.344582e+07,1,-1.414494e+09,-1.376241e+09,-1.395849e+09,-1.383135e+09,-1.359838e+09,-1.385912e+09,1.842161e+07
1,0.003416,0.001432,0.001595,0.001722,10,{'alpha': 10},-1.275815e+09,-1.426341e+09,-1.348298e+09,-1.398910e+09,...,-1.388665e+09,7.347391e+07,2,-1.414488e+09,-1.376241e+09,-1.395844e+09,-1.383129e+09,-1.359831e+09,-1.385907e+09,1.842113e+07
2,0.004099,0.000176,0.002447,0.001927,4,{'alpha': 4},-1.275842e+09,-1.426349e+09,-1.348325e+09,-1.398962e+09,...,-1.388702e+09,7.348546e+07,3,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
3,0.003550,0.001738,0.001814,0.001852,2,{'alpha': 2},-1.275851e+09,-1.426354e+09,-1.348333e+09,-1.398979e+09,...,-1.388715e+09,7.348958e+07,4,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
4,0.003086,0.001981,0.003200,0.001337,1,{'alpha': 1.0},-1.275856e+09,-1.426357e+09,-1.348338e+09,-1.398988e+09,...,-1.388722e+09,7.349165e+07,5,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
5,0.004395,0.002355,0.001677,0.001439,0.8,{'alpha': 0.8},-1.275856e+09,-1.426357e+09,-1.348339e+09,-1.398990e+09,...,-1.388723e+09,7.349206e+07,6,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
6,0.004652,0.001038,0.000604,0.000493,0.5,{'alpha': 0.5},-1.275858e+09,-1.426358e+09,-1.348340e+09,-1.398992e+09,...,-1.388725e+09,7.349268e+07,7,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
7,0.004456,0.002177,0.001892,0.001898,0.3,{'alpha': 0.3},-1.275859e+09,-1.426359e+09,-1.348341e+09,-1.398994e+09,...,-1.388726e+09,7.349311e+07,8,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
8,0.003324,0.001960,0.003013,0.001657,0.2,{'alpha': 0.2},-1.275859e+09,-1.426359e+09,-1.348342e+09,-1.398995e+09,...,-1.388727e+09,7.349332e+07,9,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07
9,0.002855,0.001520,0.002873,0.002011,0.1,{'alpha': 0.1},-1.275860e+09,-1.426359e+09,-1.348342e+09,-1.398996e+09,...,-1.388727e+09,7.349353e+07,10,-1.414487e+09,-1.376241e+09,-1.395843e+09,-1.383128e+09,-1.359830e+09,-1.385906e+09,1.842108e+07


### GridSearchCV Regresión Polinomial

In [18]:
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
from sklearn.base import BaseEstimator


In [36]:
estimator = PolynomialRegression()
degrees = np.arange(1, 10)
cv_model = GridSearchCV(estimator,
                        param_grid={'deg': degrees},
                        scoring='mean_squared_error')
cv_model.fit(X_train, y_train)

NameError: name 'PolynomialRegression' is not defined

# Ejercicio 4: Métricas sobre el conjunto de evaluación

Una vez encontrados los mejores hiperparámetros para el modelo seleccionado en los apartados anteriores se evalúa el modelo final entrenado sobre el conjunto de datos de evaluación seleccionado en el ejercicio 1. Pueden utilizar las métricas que crean convenientes.

# Informe final

Presentar un informe final con los datos inferidos del dataset utilizando herramientas gráficas y escritas teniendo en cuenta los destinatarios. En este caso colegas de la diplomatura para preparar el informe con un background adecuado al destinatario.